# **Problem Statement**

### **Business Context**

The healthcare industry is rapidly evolving, with professionals facing increasing challenges in managing vast volumes of medical data while delivering accurate and timely diagnoses. The need for quick access to comprehensive, reliable, and up-to-date medical knowledge is critical for improving patient outcomes and ensuring informed decision-making in a fast-paced environment.

Healthcare professionals often encounter information overload, struggling to sift through extensive research and data to create accurate diagnoses and treatment plans. This challenge is amplified by the need for efficiency, particularly in emergencies, where time-sensitive decisions are vital. Furthermore, access to trusted, current medical information from renowned manuals and research papers is essential for maintaining high standards of care.

To address these challenges, healthcare centers can focus on integrating systems that streamline access to medical knowledge, provide tools to support quick decision-making, and enhance efficiency. Leveraging centralized knowledge platforms and ensuring healthcare providers have continuous access to reliable resources can significantly improve patient care and operational effectiveness.

**Common Questions to Answer**

1. **Critical Care Protocols:** "What is the protocol for managing sepsis in a critical care unit?"

2. **General Surgery:** "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"

3. **Dermatology:** "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"

4. **Neurology:** "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"


### **Objective**

As an AI specialist, your task is to develop a RAG-based AI solution using renowned medical manuals to address healthcare challenges. The objective is to **understand** issues like information overload, **apply** AI techniques to streamline decision-making, **analyze** its impact on diagnostics and patient outcomes, **evaluate** its potential to standardize care practices, and **create** a functional prototype demonstrating its feasibility and effectiveness.

### **Data Description**

The **Merck Manuals** are medical references published by the American pharmaceutical company Merck & Co., that cover a wide range of medical topics, including disorders, tests, diagnoses, and drugs. The manuals have been published since 1899, when Merck & Co. was still a subsidiary of the German company Merck.

The manual is provided as a PDF with over 4,000 pages divided into 23 sections.

### **Installing and Importing Necessary Libraries and Dependencies**

In [1]:
# Install required libraries
!pip install -q langchain_community==0.3.27 \
              langchain==0.3.27 \
              chromadb==1.0.15 \
              pymupdf==1.26.3 \
              tiktoken==0.9.0 \
              ragas==0.3.0 \
              datasets==4.0.0 \
              evaluate==0.4.5 \
              langchain_openai==0.3.30

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 101.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 129.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 97.5 MB/s eta 0:00:0

**Note**:
- After running the above cell, kindly restart the runtime (for Google Colab) or notebook kernel (for Jupyter Notebook), and run all cells sequentially from the next cell.
- On executing the above line of code, you might see a warning regarding package dependencies. This error message can be ignored as the above code ensures that all necessary libraries and their dependencies are maintained to successfully execute the code in ***this notebook***.

In [1]:
# Import core libraries
import os                                                                       # Interact with the operating system (e.g., set environment variables)
import json                                                                     # Read/write JSON data
import requests  # type: ignore                                                 # Make HTTP requests (e.g., API calls); ignore type checker

# Import libraries for working with PDFs and OpenAI
from langchain.document_loaders import PyMuPDFLoader                            # Load and extract text from PDF files
# from langchain_community.document_loaders import PyPDFLoader                    # Load and extract text from PDF files
from openai import OpenAI                                                       # Access OpenAI's models and services

# Import libraries for processing dataframes and text
import tiktoken                                                                 # Tokenizer used for counting and splitting text for models
import pandas as pd                                                             # Load, manipulate, and analyze tabular data

# Import LangChain components for data loading,chunking,embedding and vector DBs
from langchain.text_splitter import RecursiveCharacterTextSplitter              # Break text into overlapping chunks for processing
from langchain.embeddings.openai import OpenAIEmbeddings                        # Create vector embeddings using OpenAI's models  # type: ignore
from langchain.vectorstores import Chroma                                       # Store and search vector embeddings using Chroma DB  # type: ignore

from datasets import Dataset                                                    # Used to structure the input (questions, answers, contexts etc.) in tabular format
from langchain_openai import ChatOpenAI                                         # This is needed since LLM is used in metric computation

# Import components to run evaluation on RAG pipeline outputs
from ragas import evaluate
from ragas.metrics import (
    Faithfulness,
    AnswerRelevancy,
    LLMContextPrecisionWithoutReference,
)

In [2]:
# Ignore Warnings
import warnings
warnings.filterwarnings('ignore')

### **Mount Google Drive**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **OpoenAI API Calling**

In [4]:
# Load the JSON file and extract values
file_name = "/content/drive/MyDrive/GenAI_Course/Codes/config.json"             # Name of the configuration file
with open(file_name, 'r') as file:                                              # Open the config file in read mode
    config = json.load(file)                                                    # Load the JSON content as a dictionary
    OPENAI_API_KEY = config.get("OPENAI_API_KEY")                               # Extract the API key from the config
    OPENAI_API_BASE = config.get("OPENAI_API_BASE")                             # Extract the OpenAI base URL from the config

# Store API credentials in environment variables
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY                                   # Set API key as environment variable
os.environ["OPENAI_BASE_URL"] = OPENAI_API_BASE                                 # Set API base URL as environment variable

# Initialize OpenAI client
client = OpenAI()                                                               # Create an instance of the OpenAI client

# **Question Answering using LLM**

### **Downloading and Loading the model using a reusable function**

In [5]:
# Define a function to get a response
def generate_response(user_input,k=5,max_tokens=500,temperature=0.3,top_p=0.95):
    try:
        # Create a chat completion using the OpenAI client
        response = client.chat.completions.create(
        model="gpt-4o-mini",                                                    # Model to use for generating responses
        messages=[
            {"role": "user", "content": user_input}                             # User_input is the input/query to respond to
        ],
        max_tokens=max_tokens,                                                  # Max number of tokens to generate in the response
        temperature=temperature,                                                # Controls randomness in output
        top_p=top_p                                                             # Controls diversity via nucleus sampling
        )
        # Extract and print the generated text from the response
        response = response.choices[0].message.content.strip()                  # Return the text content from the model's reply
    except Exception as e:
        response = f'Sorry, I encountered the following error: \n {e}'          # Return the error message in case of exception

    return response

#### **Question 1:** What is the protocol for managing sepsis in a critical care unit?

In [6]:
question_1 = "What is the protocol for managing sepsis in a critical care unit?"
base_prompt_response_1=generate_response(question_1)

#### **Question 2:** What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [7]:
question_2 = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
base_prompt_response_2=generate_response(question_2)

#### **Question 3:** What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [8]:
question_3 = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
base_prompt_response_3=generate_response(question_3)

#### **Question 4:**  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [9]:
question_4 = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
base_prompt_response_4=generate_response(question_4)

#### **Storing the generated outputs from the base prompt**

In [10]:
# Create the DataFrame
result_df = pd.DataFrame({
    "questions": [question_1, question_2, question_3, question_4],
    "base_prompt_responses": [base_prompt_response_1, base_prompt_response_2, base_prompt_response_3, base_prompt_response_4]})

# Display the DataFrame
result_df.head()

,questions,base_prompt_responses
0,What is the protocol for managing sepsis in a ...,Managing sepsis in a critical care unit involv...
1,"What are the common symptoms for appendicitis,...",Common symptoms of appendicitis include:\n\n1....
2,What are the effective treatments or solutions...,"Sudden patchy hair loss, often referred to as ..."
3,What treatments are recommended for a person w...,The treatment for a person who has sustained a...


### 👉  **Observations**


*   It is observed that The answer is derived from the context only to a limited extent.
*   E.g. in the question 1: While the context provides detailed information on antibiotic therapy, glucose management, corticosteroid use, and other supportive measures for managing sepsis, the answer includes additional elements such as early recognition, initial assessment, and specific resuscitation protocols that are not explicitly mentioned in the context. The answer does align with general sepsis management principles but extends beyond the provided context.
*   Other questions were answered correctly but providing additional unnecessary contents.



# **Question Answering using LLM with Prompt Engineering**

Now let's use prompt engineering to answer the user questions. Let's check the preformance of the model if well-defined prompt is provided.  

In [11]:
# Define a function to get a response
def generate_response(user_input,system_prompt="Answer the question", k=5,max_tokens=500,temperature=0.3,top_p=0.95):
    try:
        response = client.chat.completions.create(
        model="gpt-4o-mini",                                                    # Model to use for generating responses
        messages=[
            {"role": "system", "content": system_prompt},                       # system_prompt is the instruction for the LLM
            {"role": "user", "content": user_input}                             # User_input is the input/query to respond to
        ],
        max_tokens=max_tokens,                                                  # Max number of tokens to generate in the response
        temperature=temperature,                                                # Controls randomness in output
        top_p=top_p                                                             # Controls diversity via nucleus sampling
        )
        # Extract and print the generated text from the response
        response = response.choices[0].message.content.strip()                  # Return the text content from the model's reply
    except Exception as e:
        response = f'Sorry, I encountered the following error: \n {e}'          # Return the error message in case of exception

    return response

#### **Define 5 hyper parameter sets, Check for all the 4 questions & Store the generated outputs**

In [12]:
# Define a list of hyperparameter sets based on temperatures (randomness), top probabilities, maximum tokens to use, top n answers and find the best fit
hyperparameter_sets = [
    {"system_prompt": "Answer the question", "k": 5, "max_tokens": 500, "temperature": 0.3, "top_p": 0.95},
    {"system_prompt": "Provide a detailed medical explanation", "k": 7, "max_tokens": 600, "temperature": 0.5, "top_p": 0.9},
    {"system_prompt": "Summarize the key steps", "k": 3, "max_tokens": 400, "temperature": 0.1, "top_p": 0.8},
    {"system_prompt": "Explain like I'm a medical student", "k": 6, "max_tokens": 700, "temperature": 0.7, "top_p": 1.0},
    {"system_prompt": "Give a concise answer", "k": 4, "max_tokens": 300, "temperature": 0.2, "top_p": 0.85}
]

# List of questions
questions = [question_1, question_2, question_3, question_4]

# Initialize columns for the DataFrame
for i in range(len(hyperparameter_sets)):
    result_df[f"prompt_engg_responses_hp{i+1}"] = ""

# Loop through each question
for j, question in enumerate(questions):
    print(f"\nQuestion {j+1}:")
    # Loop through each hyperparameter set
    for i, params in enumerate(hyperparameter_sets):
        print(f"  Hyperparameter Set {i+1}:")
        # Generate response using the current hyperparameter set
        response = generate_response(user_input=question, **params)
        # Add the response to the DataFrame
        result_df.loc[j, f"prompt_engg_responses_hp{i+1}"] = response
        print(f"    {response}")

# Display the DataFrame
result_df.head()


Question 1:
  Hyperparameter Set 1:
    Managing sepsis in a critical care unit involves a systematic approach to ensure timely and effective treatment. The following protocol outlines key steps typically followed in such settings:

1. **Early Recognition**:
   - Monitor for signs and symptoms of sepsis, including fever, tachycardia, tachypnea, altered mental status, and hypotension.
   - Use screening tools like the Sequential Organ Failure Assessment (SOFA) score to assess organ dysfunction.

2. **Immediate Assessment**:
   - Obtain a thorough history and physical examination.
   - Identify potential sources of infection (e.g., urinary tract, pneumonia, abdominal infections).

3. **Laboratory and Diagnostic Tests**:
   - Obtain blood cultures before starting antibiotics.
   - Perform relevant laboratory tests (e.g., complete blood count, lactate levels, renal function tests).
   - Consider imaging studies to identify the source of infection.

4. **Fluid Resuscitation**:
   - Adminis

,questions,base_prompt_responses,prompt_engg_responses_hp1,prompt_engg_responses_hp2,prompt_engg_responses_hp3,prompt_engg_responses_hp4,prompt_engg_responses_hp5
0,What is the protocol for managing sepsis in a ...,Managing sepsis in a critical care unit involv...,Managing sepsis in a critical care unit involv...,Sepsis is a life-threatening condition that ar...,The protocol for managing sepsis in a critical...,Managing sepsis in a critical care unit is cru...,The protocol for managing sepsis in a critical...
1,"What are the common symptoms for appendicitis,...",Common symptoms of appendicitis include:\n\n1....,Common symptoms of appendicitis include:\n\n1....,Appendicitis is an inflammation of the appendi...,Common symptoms of appendicitis include:\n\n1....,Sure! Let’s break this down.\n\n### Common Sym...,Common symptoms of appendicitis include abdomi...
2,What are the effective treatments or solutions...,"Sudden patchy hair loss, often referred to as ...","Sudden patchy hair loss, often referred to as ...","Sudden patchy hair loss, often referred to as ...",**Key Steps for Addressing Sudden Patchy Hair ...,"Sudden patchy hair loss, often referred to as ...",Effective treatments for sudden patchy hair lo...
3,What treatments are recommended for a person w...,The treatment for a person who has sustained a...,Treatment for a person who has sustained a phy...,When a person sustains a physical injury to br...,The treatment for a person who has sustained a...,When treating a person who has sustained a phy...,Treatment for brain injury may include:\n\n1. ...


### 👉  **Observations**


*   I have tried 5 different hyper parameters to do the prompt engineering based model training.
*   It is observed that The answer is derived from the context only to a limited extent.  E.g. in the question 1: While the context provides detailed information on antibiotic therapy, glucose management, corticosteroid use, and other supportive measures for managing sepsis, the answer includes additional elements such as early recognition, initial assessment, and specific resuscitation protocols that are not explicitly mentioned in the context. The answer does align with general sepsis management principles but extends beyond the provided context.
*   It is observed that Hyper Parameter 5 is producing kind of better result as compared to other 4. It is not generating unwanted contents as comapred to other 4. Still it has moderate relevance. Let's try for RAG.



# **Data Preparation for RAG**

### **Loading the Data**

In [13]:
# Load the data from the Google Drive Path using PyMuPDFLoader Library
pdf_Source_path = "/content/drive/MyDrive/GenAI_Course/Codes/medical_diagnosis_manual.pdf"
pdf_loader = PyMuPDFLoader(pdf_Source_path)
user_manual = pdf_loader.load()

### **Data Overview**

In [14]:
# Show the first 5 pages of the PDF
for i in range(5):
    print(f"Page Number : {i+1}",end="\n")
    print(user_manual[i].page_content,end="\n")

Page Number : 1
jagatycet@gmail.com
QZPEX7L8AV
meant for personal use by jagatycet@gm
shing the contents in part or full is liable
Page Number : 2
jagatycet@gmail.com
QZPEX7L8AV
This file is meant for personal use by jagatycet@gmail.com only.
Sharing or publishing the contents in part or full is liable for legal action.
Page Number : 3
Table of Contents
1
Front    ................................................................................................................................................................................................................
1
Cover    .......................................................................................................................................................................................................
2
Front Matter    ...........................................................................................................................................................................................
53
1 - 

### 👉  **Observations**

As the first 5 pages contains mostly the index table of content. Let's explore some real data pages. Page 53 onwards real research content. First 10 pages are table of content.

In [15]:
for i in range(62, 67): # Loop for pages 63 to 67
    print(f"Page Number : {i+1}",end="\n")
    print(user_manual[i].page_content,end="\n")

Page Number : 63
1 - Nutritional Disorders
Chapter 1. Nutrition: General Considerations
Introduction
Nutrition is the science of food and its relationship to health. Nutrients are chemicals in foods that are
used by the body for growth, maintenance, and energy. Nutrients that cannot be synthesized by the body
and thus must be derived from the diet are considered essential. They include vitamins, minerals, some
amino acids, and some fatty acids. Nutrients that the body can synthesize from other compounds,
although they may also be derived from the diet, are considered nonessential. Macronutrients are
required by the body in relatively large amounts; micronutrients are needed in minute amounts.
Lack of nutrients can result in deficiency syndromes (eg, kwashiorkor, pellagra) or other disorders (see p.
9). Excess intake of macronutrients can lead to obesity (see p. 56) and related disorders; excess intake of
micro-nutrients can be toxic. Also, the balance of various types of nutrients, suc

### **Data Chunking**


*   Chunk the PDF into Text sections using Token-based splitter
*   Split the loaded PDF into chunks
*   Check the number of chunks created



In [16]:
# Splitting of Text to chunk the data
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name='cl100k_base',
    chunk_size=512,
    chunk_overlap=20
)
# Based on above chunks split and load the PDF
document_chunks = pdf_loader.load_and_split(text_splitter)
# Check the number of chunks
len(document_chunks)

8469

Let's take a look at consecutive chunks from the document to see the overlap of contents.Here I am checking the same first research content page which is the page 63 of the document. Code assigns it as page 62 as it starts with index 0.

In [17]:
i = 80
document_chunks[i]

Document(metadata={'producer': 'pdf-lib (https://github.com/Hopding/pdf-lib)', 'creator': 'Atop CHM to PDF Converter', 'creationdate': '2012-06-15T05:44:40+00:00', 'source': '/content/drive/MyDrive/GenAI_Course/Codes/medical_diagnosis_manual.pdf', 'file_path': '/content/drive/MyDrive/GenAI_Course/Codes/medical_diagnosis_manual.pdf', 'total_pages': 4114, 'format': 'PDF 1.7', 'title': 'The Merck Manual of Diagnosis & Therapy, 19th Edition', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-11-01T23:47:50+00:00', 'trapped': '', 'modDate': 'D:20251101234750Z', 'creationDate': 'D:20120615054440Z', 'page': 62}, page_content='1 - Nutritional Disorders\nChapter 1. Nutrition: General Considerations\nIntroduction\nNutrition is the science of food and its relationship to health. Nutrients are chemicals in foods that are\nused by the body for growth, maintenance, and energy. Nutrients that cannot be synthesized by the body\nand thus must be derived from the diet are considered essentia

In [18]:
document_chunks[i+1]

Document(metadata={'producer': 'pdf-lib (https://github.com/Hopding/pdf-lib)', 'creator': 'Atop CHM to PDF Converter', 'creationdate': '2012-06-15T05:44:40+00:00', 'source': '/content/drive/MyDrive/GenAI_Course/Codes/medical_diagnosis_manual.pdf', 'file_path': '/content/drive/MyDrive/GenAI_Course/Codes/medical_diagnosis_manual.pdf', 'total_pages': 4114, 'format': 'PDF 1.7', 'title': 'The Merck Manual of Diagnosis & Therapy, 19th Edition', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-11-01T23:47:50+00:00', 'trapped': '', 'modDate': 'D:20251101234750Z', 'creationDate': 'D:20120615054440Z', 'page': 62}, page_content='—see\nTable 1-1). However, the actual rate of increase also depends on what foods are consumed with the\ncarbohydrate.\nCarbohydrates with a high glycemic index may increase plasma glucose to high levels rapidly. It is\nhypothesized that, as a result, insulin levels increase, inducing hypoglycemia and hunger, which tends to\nlead to consumption of excess calo

### Embedding

In [20]:
# Initialize the OpenAI Embeddings model with API credentials
embedding_model = OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY,                                              # The OpenAI API key for authentication
    openai_api_base=OPENAI_API_BASE,                                            # The OpenAI API base URL endpoint
    chunk_size=512                                                              # Set a smaller chunk size for the embedding model to handle token limits
)

# Generate embeddings (vector representations) for the first two document chunks
embedding_1 = embedding_model.embed_query(document_chunks[0].page_content)      # Embedding for chunk 0
embedding_2 = embedding_model.embed_query(document_chunks[1].page_content)      # Embedding for chunk 1

# Check and print the dimension (length) of the embedding vector
print("Dimension of the embedding vector ", len(embedding_1))                   # Typically 1536 or 2048 depending on model

Dimension of the embedding vector  1536


### 👉  **Observations**

Providing the chunks of the entrie document was exceeding the maximum limit of OpenAI embedddings to be stored in the Vector Store. So applied Chunk Size.

In [21]:
# Verify if both embeddings have the same dimension (should be True)
len(embedding_1) == len(embedding_2)

# Return/display the two embedding vectors for further inspection or use
embedding_1, embedding_2

([-0.007404245811302993,
  0.005111295523161028,
  0.007237232656858609,
  -0.011280340714028947,
  -0.011182915985885297,
  0.03081390976552033,
  -0.028294795694917045,
  -0.025218972351262647,
  -0.028378301806477928,
  -0.015685309519285724,
  0.043005863520702,
  0.019136914090254576,
  0.0055914579929426484,
  0.020626113863671264,
  0.00826018718009251,
  0.01446054762845707,
  0.023423581639477484,
  -0.0006680522685315525,
  0.008072297497757907,
  -0.005382691782717824,
  -0.0066422488180194654,
  0.023047802274808276,
  -0.002447785235153065,
  0.035573782338878715,
  -0.01777297348417921,
  -0.010953273247770252,
  0.026179296359503265,
  -0.021656026298212616,
  -0.010730589352285281,
  -0.004331901423302318,
  0.0007111102819281234,
  0.0024095113678570057,
  -0.02505195826549564,
  -0.02638806350105071,
  -0.010048619049247597,
  -0.004460640477619986,
  0.015003340147570663,
  -0.003858697563320115,
  0.011816174390512465,
  0.006951918712041666,
  0.011621325865547788,

# **Vector Database**

### **Create Vector Database Directory**

In [22]:
out_dir = 'Healthcare_db'

if not os.path.exists(out_dir):
  os.makedirs(out_dir)

#### **Create Vector Database from Documents**

In [23]:
# Building the vector store and saving it to disk for future use
vectorstore = Chroma.from_documents(
    document_chunks,                                                            # Documents to index
    embedding_model,                                                            # Embedding model for converting text to vectors
    persist_directory=out_dir)                                                  # Save vector DB files here

#### **Load Vector Database**

In [24]:
vectorstore = Chroma(
    persist_directory=out_dir,
    embedding_function=embedding_model
)

#### **Vector Database Data Exploration**

In [25]:
vectorstore.embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7f6e00df9ee0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7f6dfb20c6e0>, model='text-embedding-ada-002', deployment='text-embedding-ada-002', openai_api_version='', openai_api_base='https://aibe.mygreatlearning.com/openai/v1', openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key='gl-U2FsdGVkX19CKpk7ptTqNpW/m0jPtkM9MnpK+claxFXde4ITkg4YI/MSkAaKLXbA', openai_organization=None, allowed_special=set(), disallowed_special='all', chunk_size=512, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None)

In [26]:
vectorstore.similarity_search("What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?",k=4)

[Document(metadata={'subject': '', 'total_pages': 4114, 'page': 3412, 'creationdate': '2012-06-15T05:44:40+00:00', 'author': '', 'modDate': 'D:20251101234750Z', 'creator': 'Atop CHM to PDF Converter', 'source': '/content/drive/MyDrive/GenAI_Course/Codes/medical_diagnosis_manual.pdf', 'moddate': '2025-11-01T23:47:50+00:00', 'title': 'The Merck Manual of Diagnosis & Therapy, 19th Edition', 'producer': 'pdf-lib (https://github.com/Hopding/pdf-lib)', 'creationDate': 'D:20120615054440Z', 'file_path': '/content/drive/MyDrive/GenAI_Course/Codes/medical_diagnosis_manual.pdf', 'format': 'PDF 1.7', 'keywords': '', 'trapped': ''}, page_content='support groups may provide assistance to the families of brain-injured patients.\nFor patients whose coma exceeds 24 h, 50% of whom have major persistent neurologic sequelae, a\nprolonged period of rehabilitation, particularly in cognitive and emotional areas, is often required.\nRehabilitation services should be planned early.\nThe Merck Manual of Diagnos

### **Retriever**

Created a retriever that can query an input text and retrieve the top-k documents that are most relevant from the vector store.

- Under the hood, a similarity score is computed between the embedded query and all the chunks in the database
- The top k chunks with the highest similarity scores are then returned.

In [27]:
retriever = vectorstore.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 4}
)

Let's ask a simple query and see what document chunks are returned based on the similarity search.

In [28]:
user_input = "Who are the authors of this article and who published this article ?"

relevant_document_chunks = retriever.get_relevant_documents(user_input)

print('len(relevant_document_chunks) : ', len(relevant_document_chunks))
print('\n\n')
for document in relevant_document_chunks:
    print(document.page_content)

print('len(relevant_document_chunks[0].page_content) : ', len(relevant_document_chunks[0].page_content))


len(relevant_document_chunks) :  4



Medical College
Senior Assistant Editor
JUSTIN L. KAPLAN, MD
Merck & Co., Inc, and Clinical Associate Professor, Department of Emergency Medicine, Jefferson
Medical College
Editorial Board
RICHARD K. ALBERT, MD
Professor, Department of Medicine, University
of Colorado Health Sciences Center; Chief of
Medicine, Denver Health Medical Center
MARJORIE A. BOWMAN, MD, MPA
Professor and Founding Chair of Family
Practice and Community Health, University of
Pennsylvania School of Medicine
GLENN D. BRAUNSTEIN, MD
Professor of Medicine, David Geffen School
of Medicine at University of California, Los
Angeles; Chairman, Department of Medicine,
Cedars-Sinai Medical Center
SIDNEY COHEN, MD
Professor of Medicine and Director, Research
Programs, Thomas Jefferson University School
of Medicine
LINDA EMANUEL, PhD
Professor of Medicine, Northwestern University,
Feinberg School of Medicine, Buehler Center
on Aging
JAN FAWCETT, MD
Professor of Psychiatry, University of 

It can be observed that the chunks are related to the user query and can perhaps contain the answer.

#### **Designing the System Prompt**

System Prompt designing is a crucial part of designing a RAG based system, it consists mainly of two parts:

- system message: This is the instruction that has to be given to the LLM.
- user message template: This is a message template that contains the context retrieved from the document chunks and the User Query.

In [29]:
qna_system_message = """
You are an assistant whose work is to give answers to questions with repect to a context.
User input will have the context required by you to answer user questions.

This context will begin with the token: ###Context.
The context contains references to specific portions of a document relevant to the user query.

User questions will begin with the token: ###Question.

Strictly answer only using the information provided in the ###Context.
Do not mention anything about the information in ###Context or the question in ###Question in your final answer.

If the answer to ###Question cannot be derived from the ###Context, just respond by saying "I don't know".

Remember that the answer to ###Question might not always be directly present in the information provided in the ###Context.
the answer can be indirectly derived from the information in ###Context.

"""

**Note**: It is important to specify that the LLM should not attempt to answer the question if the context provided (retrieved from the knowledge base provided) doesn't contain the information required. We don't want the LLM to use the knowledge from its training data and/or hallucinate to share a "seemingly correct" answer.

#### **Designing the User Prompt**

In [30]:
qna_user_message_template = """
Conider the following ###Context and ###Question
###Context
{context}

###Question
{question}
"""

### **Response Function**

In [31]:
def generate_rag_response(user_input,k=5,max_tokens=500,temperature=0.3,top_p=0.95):
    global qna_system_message,qna_user_message_template
    # Retrieve relevant document chunks
    relevant_document_chunks = retriever.get_relevant_documents(query=user_input,k=k)
    context_list = [d.page_content for d in relevant_document_chunks]

    # Combine document chunks into a single context
    context_for_query = ". ".join(context_list)

    user_message = qna_user_message_template.replace('{context}', context_for_query)
    user_message = user_message.replace('{question}', user_input)

    # Generate the response
    try:
        response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": qna_system_message},
            {"role": "user", "content": user_message}
        ],
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p
        )
        # Extract and print the generated text from the response
        response = response.choices[0].message.content.strip()
    except Exception as e:
        response = f'Sorry, I encountered the following error: \n {e}'

    return response

## **Question Answering using RAG and store the results**

#### **Question 1:** What is the protocol for managing sepsis in a critical care unit?
#### **Question 2:** What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?
#### **Question 3:** What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?
#### **Question 4:**  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [33]:
# Define 5 sets of hyperparameters to find the best fit
param_sets = [
    {'k': 3,  'max_tokens': 300, 'temperature': 0.2, 'top_p': 0.9},
    {'k': 5,  'max_tokens': 500, 'temperature': 0.3, 'top_p': 0.95},
    {'k': 7,  'max_tokens': 500, 'temperature': 0.4, 'top_p': 0.9},
    {'k': 5,  'max_tokens': 600, 'temperature': 0.5, 'top_p': 0.85},
    {'k': 10, 'max_tokens': 700, 'temperature': 0.7, 'top_p': 0.9}
]

# Single for-loop to generate RAG responses for each parameter set
for i, params in enumerate(param_sets, start=1):
    result_df[f'responses_with_RAG_set_{i}'] = [
        generate_rag_response(q, **params) for q in result_df['questions']
    ]

# Display the DataFrame with all results
result_df.head()

,questions,base_prompt_responses,prompt_engg_responses_hp1,prompt_engg_responses_hp2,prompt_engg_responses_hp3,prompt_engg_responses_hp4,prompt_engg_responses_hp5,responses_with_RAG_set_1,responses_with_RAG_set_2,responses_with_RAG_set_3,responses_with_RAG_set_4,responses_with_RAG_set_5
0,What is the protocol for managing sepsis in a ...,Managing sepsis in a critical care unit involv...,Managing sepsis in a critical care unit involv...,Sepsis is a life-threatening condition that ar...,The protocol for managing sepsis in a critical...,Managing sepsis in a critical care unit is cru...,The protocol for managing sepsis in a critical...,The protocol for managing sepsis in a critical...,The protocol for managing sepsis in a critical...,The protocol for managing sepsis in a critical...,The protocol for managing sepsis in a critical...,The protocol for managing sepsis in a critical...
1,"What are the common symptoms for appendicitis,...",Common symptoms of appendicitis include:\n\n1....,Common symptoms of appendicitis include:\n\n1....,Appendicitis is an inflammation of the appendi...,Common symptoms of appendicitis include:\n\n1....,Sure! Let’s break this down.\n\n### Common Sym...,Common symptoms of appendicitis include abdomi...,The common symptoms of appendicitis include ep...,Common symptoms of appendicitis include epigas...,Common symptoms of appendicitis include epigas...,The common symptoms of appendicitis include ep...,The common symptoms of appendicitis include ep...
2,What are the effective treatments or solutions...,"Sudden patchy hair loss, often referred to as ...","Sudden patchy hair loss, often referred to as ...","Sudden patchy hair loss, often referred to as ...",**Key Steps for Addressing Sudden Patchy Hair ...,"Sudden patchy hair loss, often referred to as ...",Effective treatments for sudden patchy hair lo...,Effective treatments for sudden patchy hair lo...,Effective treatments for sudden patchy hair lo...,Effective treatments for sudden patchy hair lo...,Effective treatments for sudden patchy hair lo...,Effective treatments for sudden patchy hair lo...
3,What treatments are recommended for a person w...,The treatment for a person who has sustained a...,Treatment for a person who has sustained a phy...,When a person sustains a physical injury to br...,The treatment for a person who has sustained a...,When treating a person who has sustained a phy...,Treatment for brain injury may include:\n\n1. ...,Initial treatment consists of ensuring a relia...,Initial treatment consists of ensuring a relia...,Initial treatment consists of ensuring a relia...,Initial treatment consists of ensuring a relia...,Initial treatment for a physical injury to bra...


### 👉  **Observations**


*   Here I have tried to ask the same set of 4 questions through base prompt, 5 different hyper parameterized prompt enginnering and 5 different hyper parameterized RAG based approach and consolidated the results in a dataframe for comparison.
*   From the answers it is observed that the best results were received from RAG then Prompt engineering with Hyper_Parameter_5 (as compared to other 3 hyper parameters) and lastly base prompt.

*   From RAG response it looks like the Answer is completely derived from the Context provided. E.g. for question 1: Each step in the protocol for managing sepsis in a critical care unit mentioned in the Answer is directly supported by the information in the Context. The answer covers antibiotic therapy, fluid resuscitation, supportive care, monitoring, blood glucose control, corticosteroid therapy, surgical intervention, prognosis, and additional treatments like activated protein C, all of which are detailed in the Context.

*   Whereas for other methods it is observed that The answer is derived from the context only to a limited extent. E.g. in the question 1: While the context provides detailed information on antibiotic therapy, glucose management, corticosteroid use, and other supportive measures for managing sepsis, the answer includes additional elements such as early recognition, initial assessment, and specific resuscitation protocols that are not explicitly mentioned in the context. The answer does align with general sepsis management principles but extends beyond the provided context.



## **Output Evaluation**

**We will use the following evaluation metrices**

* **Faithfulness** - Checks whether the generated response stays true to the retrieved context without adding unsupported or hallucinated information.
* **Answer Relevancy** - Measures how directly the response addresses the user's query, ensuring that the answer is not only correct but also useful.
* **Context Precision** - Evaluates how precisely the retrieved context contributes to answering the query, reducing noise and irrelevant details.

In [34]:
# Prompt to evaluate how relevant the answer is to the original question
relevance_rater_system_message = """
You will be presented with a ###Question, the ###Context used by the AI system to generate a response, and the AI-generated ###Answer.

Your task is to judge the extent to which the ###Answer is relevant to the ###Question, considering whether it directly addresses the key aspects of the ###Question based on the provided ###Context.

Rate the relevance as follows:
- Rate 1 – The ###Answer is not relevant to the ###Question at all.
- Rate 2 – The ###Answer is only slightly relevant to the **###Question**, missing key aspects.
- Rate 3 – The ###Answer is moderately relevant, addressing some parts of the **###Question** but leaving out important details.
- Rate 4 – The ###Answer is mostly relevant, covering key aspects but with minor gaps.
- Rate 5 – The ###Answer is fully relevant, directly answering all important aspects of the **###Question** with appropriate details from the **###Context**.

The final output should be a single overall rating in the range of 1 to 5, along with a brief explanation of the rationale for the rating.
"""

# Prompt to evaluate how well the answer is grounded in the provided context
groundedness_rater_system_message = """
You will be presented a ###Question, ###Context used by the AI system and AI generated ###Answer.

Your task is to judge the extent to which the ###Answer is derived from ###Context.

Rate it 1 - if The ###Answer is not derived from the ###Context at all
Rate it 2 - if The ###Answer is derived from the ###Context only to a limited extent
Rate it 3 - if The ###Answer is derived from ###Context to a good extent
Rate it 4 - if The ###Answer is derived from ###Context mostly
Rate it 5 - if The ###Answer is is derived from ###Context completely

The final output should be a single overall rating in the range of 1 to 5, along with a brief explanation of the rationale for the rating.
"""

In [35]:
user_message_template = """
###Question
{question}

###Context
{context}

###Answer
{answer}
"""

### **Define Evaluator LLM function**

In [36]:
def generate_ground_relevance_response(user_input,response,  max_tokens=500, temperature=0.3, top_p=0.95):
    global qna_user_message_template

    context_for_query = [doc.page_content for doc in retriever.get_relevant_documents(user_input, k=5)]

    # Combine user_prompt and system_message to create the prompt
    groundedness_prompt = f"""[INST]{groundedness_rater_system_message}\n
                {'user'}: {user_message_template.format(context=context_for_query, question=user_input, answer=response)}
                [/INST]"""

    # Combine user_prompt and system_message to create the prompt
    relevance_prompt = f"""[INST]{relevance_rater_system_message}\n
                {'user'}: {user_message_template.format(context=context_for_query, question=user_input, answer=response)}
                [/INST]"""

    response_1 = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "user", "content": groundedness_prompt}
                ],
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p
            )

    response_2 = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "user", "content": relevance_prompt}
                ],
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p
            )

    return response_1.choices[0].message.content,response_2.choices[0].message.content

#### **Evaluation 1: Base Prompt Response Evaluation**

In [41]:
# Single for loop for Base Prompt evaluation of 4 questions
for i in range(len(result_df)):
    ground, rel = generate_ground_relevance_response(
        user_input=result_df.questions[i],
        response=result_df.base_prompt_responses[i],
        max_tokens=512
    )
    print(f"Question {i+1} Results:\n")
    print(ground, end="\n\n")
    print(rel, end="\n\n" + "="*60 + "\n\n")

Question 1 Results:

3 - The answer is derived from the context to a good extent. The context provides detailed information about the management of sepsis, including the use of antibiotics, fluid resuscitation, and monitoring, which are reflected in the answer. However, the answer includes additional details such as the use of screening tools (SOFA, qSOFA) and specific initial interventions (fluid resuscitation within the first hour), which are not explicitly mentioned in the context. Therefore, while the answer aligns well with the context, it also incorporates some information not directly sourced from it.

Rate 3 – The ###Answer is moderately relevant, addressing some parts of the ###Question but leaving out important details.

Explanation: The answer provides a general overview of the protocol for managing sepsis in a critical care unit, including early recognition, initial assessment, immediate interventions, monitoring, and source control. However, it lacks specific details from 

#### **Evaluation 2: Prompt Engineering Response Evaluation**

In [48]:
# Create a nested loop to eavaluate Prompt Engineering Responses for each hyperparameter set and each question
for i in range(1, 6):  # Iterate from hyperparameter set 1 to 5
    hp_column_name = f"prompt_engg_responses_hp{i}"
    print(f"\n" + "="*80)
    print(f"--- Evaluating Responses for {hp_column_name} ---")
    print("="*80 + "\n")

    for j in range(len(result_df)):  # Iterate through each question (0 to 3)
        current_question = result_df.questions[j]
        current_response = result_df[hp_column_name][j]

        print(f"Question {j+1}: {current_question}\n")
        print(f"Response from {hp_column_name}:\n{current_response}\n")

        # Call generate_ground_relevance_response
        ground, rel = generate_ground_relevance_response(
            user_input=current_question,
            response=current_response,
            max_tokens=512  # Using the max_tokens value from previous cells
        )

        print("Groundedness Evaluation:\n", ground)
        print("\nRelevance Evaluation:\n", rel)
        print("\n" + "-"*80 + "\n")


--- Evaluating Responses for prompt_engg_responses_hp1 ---

Question 1: What is the protocol for managing sepsis in a critical care unit?

Response from prompt_engg_responses_hp1:
Managing sepsis in a critical care unit involves a systematic approach to ensure timely and effective treatment. The following protocol outlines key steps typically followed in such settings:

1. **Early Recognition**:
   - Monitor for signs and symptoms of sepsis, including fever, tachycardia, tachypnea, altered mental status, and hypotension.
   - Use screening tools like the Sequential Organ Failure Assessment (SOFA) score to assess organ dysfunction.

2. **Immediate Assessment**:
   - Obtain a thorough history and physical examination.
   - Identify potential sources of infection (e.g., urinary tract, pneumonia, abdominal infections).

3. **Laboratory and Diagnostic Tests**:
   - Obtain blood cultures before starting antibiotics.
   - Perform relevant laboratory tests (e.g., complete blood count, lactate

#### **Evaluation 3: RAG Response Evaluation**

In [49]:
# Create a nested loop to eavaluate RAG response for each hyperparameter set and each question
for i in range(1, 6):  # Iterate from hyperparameter set 1 to 5
    hp_column_name = f"responses_with_RAG_set_{i}"
    print(f"\n" + "="*80)
    print(f"--- Evaluating Responses for {hp_column_name} ---")
    print("="*80 + "\n")

    for j in range(len(result_df)):  # Iterate through each question (0 to 3)
        current_question = result_df.questions[j]
        current_response = result_df[hp_column_name][j]

        print(f"Question {j+1}: {current_question}\n")
        print(f"Response from {hp_column_name}:\n{current_response}\n")

        # Call generate_ground_relevance_response
        ground, rel = generate_ground_relevance_response(
            user_input=current_question,
            response=current_response,
            max_tokens=512  # Using the max_tokens value from previous cells
        )

        print("Groundedness Evaluation:\n", ground)
        print("\nRelevance Evaluation:\n", rel)
        print("\n" + "-"*80 + "\n")


--- Evaluating Responses for responses_with_RAG_set_1 ---

Question 1: What is the protocol for managing sepsis in a critical care unit?

Response from responses_with_RAG_set_1:
The protocol for managing sepsis in a critical care unit includes the following steps:

1. **Specimen Collection**: Parenteral antibiotics should be administered after obtaining specimens of blood, body fluids, and wound sites for Gram stain and culture.

2. **Empiric Therapy**: Initiate very prompt empiric therapy immediately upon suspecting sepsis, as it is essential and may be lifesaving.

3. **Antibiotic Selection**: Choose antibiotics based on the suspected source, clinical setting, knowledge or suspicion of causative organisms, sensitivity patterns, and previous culture results.

4. **Antibiotic Regimens**: 
   - For septic shock of unknown cause, consider gentamicin or tobramycin plus a 3rd-generation cephalosporin (e.g., cefotaxime, ceftriaxone, or ceftazidime if Pseudomonas is suspected).
   - Alterna

### 👉  **Observations**



*   For the base prompt response it is observed that the Answer is moderately relevant, addressing some parts of the Question but leaving out important details. Few questions were answered as per the context with proper precision.

*   For the prompt engineering response it is observed that some cases it lacks specific details from the provided context. Few questions were answered as per the context with proper precision. Some cases it generated unwanted extra information.

*   For the RAG response it is observed that all the questions were answered as per the context with proper precision with highest scores.

To be double sure let's do the evaluation usinf RAGAS as well.


## **Evaluating Responses using RAGAS**

### **Evaluation 1:** Base Prompt Response Evaluation

In [50]:
# Initialize the evaluator LLM
evaluator_llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# Initialize evaluation metrics of RAGAS module
faithfulness = Faithfulness()
answer_relevancy = AnswerRelevancy()
context_precision = LLMContextPrecisionWithoutReference()

In [59]:
# List of user questions
RAGAS_questions = [result_df.questions[0],
             result_df.questions[1],
             result_df.questions[2],
             result_df.questions[3]]

# List of Responses from Base Model
RAGAS_responses_with_base = [result_df.base_prompt_responses[0],
                       result_df.base_prompt_responses[1],
                       result_df.base_prompt_responses[2],
                       result_df.base_prompt_responses[3]]

# Retrieve top-6 documents as context for each question
RAGAS_contexts = [
    [doc.page_content for doc in retriever.get_relevant_documents(q, k=6)]
    for q in RAGAS_questions
]

In [60]:
# Wrap into HuggingFace Dataset
ragas_dataset_with_RAG = Dataset.from_dict({
    "question": RAGAS_questions,
    "answer": RAGAS_responses_with_base,
    "contexts": RAGAS_contexts,
    "reference": RAGAS_questions
})

# Run RAGAS evaluation
result_with_rag = evaluate(
    ragas_dataset_with_RAG,
    metrics=[
        answer_relevancy,
        context_precision,
        faithfulness,
    ],
    llm=evaluator_llm,
    embeddings=embedding_model
)

# Convert results to DataFrame
df_rag = result_with_rag.to_pandas()
df_rag

Evaluating:   0%|          | 0/12 [00:00<?, ?it/s]

,user_input,retrieved_contexts,response,reference,answer_relevancy,llm_context_precision_without_reference,faithfulness
0,What is the protocol for managing sepsis in a ...,[Parenteral antibiotics should be given after ...,Managing sepsis in a critical care unit involv...,What is the protocol for managing sepsis in a ...,1.000000,0.700000,0.600000
1,"What are the common symptoms for appendicitis,...",[Appendicitis\nAppendicitis is acute inflammat...,Common symptoms of appendicitis include:\n\n1....,"What are the common symptoms for appendicitis,...",0.911975,1.000000,0.562500
2,What are the effective treatments or solutions...,"[corticosteroids, retinoids, or immunosuppress...","Sudden patchy hair loss, often referred to as ...",What are the effective treatments or solutions...,0.944186,0.700000,0.612903
3,What treatments are recommended for a person w...,[support groups may provide assistance to the ...,The treatment for a person who has sustained a...,What treatments are recommended for a person w...,0.935217,0.583333,0.681818


### 👉  **Observations**

The answers for all 4 questions look like relevant but the faithfulness is very low in the base prompt model.

#### **Evaluation 2:** Prompt Engineering Response Evaluation

In [61]:
all_evaluation_results = {}

# Loop through each hyperparameter set (hp1 to hp5)
for i in range(1, 6):
    hp_column_name = f"prompt_engg_responses_hp{i}"
    print(f"------------ Evaluating {hp_column_name} ------------")

    # Extract responses for the current hyperparameter set
    current_responses = result_df[hp_column_name].tolist()

    # Create a HuggingFace Dataset for the current set
    ragas_dataset_hp = Dataset.from_dict({
        "question": RAGAS_questions,
        "answer": current_responses,
        "contexts": RAGAS_contexts,
        "reference": RAGAS_questions
    })

    # Run RAGAS evaluation
    result_hp = evaluate(
        ragas_dataset_hp,
        metrics=[
            answer_relevancy,
            context_precision,
            faithfulness,
        ],
        llm=evaluator_llm,
        embeddings=embedding_model
    )

    # Convert results to DataFrame
    df_hp = result_hp.to_pandas()

    # Store the DataFrame in the dictionary
    all_evaluation_results[hp_column_name] = df_hp

    # Print the DataFrame
    print(df_hp)

    # Calculate and print average metrics
    print("\nAverage metrics for current hyperparameter set:")
    print(df_hp.mean(numeric_only=True))
    print("-" * 50)


------------ Evaluating prompt_engg_responses_hp1 ------------


Evaluating:   0%|          | 0/12 [00:00<?, ?it/s]

                                          user_input  \
0  What is the protocol for managing sepsis in a ...   
1  What are the common symptoms for appendicitis,...   
2  What are the effective treatments or solutions...   
3  What treatments are recommended for a person w...   

                                  retrieved_contexts  \
0  [Parenteral antibiotics should be given after ...   
1  [Appendicitis\nAppendicitis is acute inflammat...   
2  [corticosteroids, retinoids, or immunosuppress...   
3  [support groups may provide assistance to the ...   

                                            response  \
0  Managing sepsis in a critical care unit involv...   
1  Common symptoms of appendicitis include:\n\n1....   
2  Sudden patchy hair loss, often referred to as ...   
3  Treatment for a person who has sustained a phy...   

                                           reference  answer_relevancy  \
0  What is the protocol for managing sepsis in a ...          1.000000   
1  What a

Evaluating:   0%|          | 0/12 [00:00<?, ?it/s]

                                          user_input  \
0  What is the protocol for managing sepsis in a ...   
1  What are the common symptoms for appendicitis,...   
2  What are the effective treatments or solutions...   
3  What treatments are recommended for a person w...   

                                  retrieved_contexts  \
0  [Parenteral antibiotics should be given after ...   
1  [Appendicitis\nAppendicitis is acute inflammat...   
2  [corticosteroids, retinoids, or immunosuppress...   
3  [support groups may provide assistance to the ...   

                                            response  \
0  Sepsis is a life-threatening condition that ar...   
1  Appendicitis is an inflammation of the appendi...   
2  Sudden patchy hair loss, often referred to as ...   
3  When a person sustains a physical injury to br...   

                                           reference  answer_relevancy  \
0  What is the protocol for managing sepsis in a ...          0.988421   
1  What a

...

                                          user_input  \
0  What is the protocol for managing sepsis in a ...   
1  What are the common symptoms for appendicitis,...   
2  What are the effective treatments or solutions...   
3  What treatments are recommended for a person w...   

                                  retrieved_contexts  \
0  [Parenteral antibiotics should be given after ...   
1  [Appendicitis\nAppendicitis is acute inflammat...   
2  [corticosteroids, retinoids, or immunosuppress...   
3  [support groups may provide assistance to the ...   

                                            response  \
0  The protocol for managing sepsis in a critical...   
1  Common symptoms of appendicitis include:\n\n1....   
2  **Key Steps for Addressing Sudden Patchy Hair ...   
3  The treatment for a person who has sustained a...   

                                           reference  answer_relevancy  \
0  What is the protocol for managing sepsis in a ...          1.000000   
1  What a

Evaluating:   0%|          | 0/12 [00:00<?, ?it/s]

                                          user_input  \
0  What is the protocol for managing sepsis in a ...   
1  What are the common symptoms for appendicitis,...   
2  What are the effective treatments or solutions...   
3  What treatments are recommended for a person w...   

                                  retrieved_contexts  \
0  [Parenteral antibiotics should be given after ...   
1  [Appendicitis\nAppendicitis is acute inflammat...   
2  [corticosteroids, retinoids, or immunosuppress...   
3  [support groups may provide assistance to the ...   

                                            response  \
0  Managing sepsis in a critical care unit is cru...   
1  Sure! Let’s break this down.\n\n### Common Sym...   
2  Sudden patchy hair loss, often referred to as ...   
3  When treating a person who has sustained a phy...   

                                           reference  answer_relevancy  \
0  What is the protocol for managing sepsis in a ...          0.975360   
1  What a

Evaluating:   0%|          | 0/12 [00:00<?, ?it/s]

                                          user_input  \
0  What is the protocol for managing sepsis in a ...   
1  What are the common symptoms for appendicitis,...   
2  What are the effective treatments or solutions...   
3  What treatments are recommended for a person w...   

                                  retrieved_contexts  \
0  [Parenteral antibiotics should be given after ...   
1  [Appendicitis\nAppendicitis is acute inflammat...   
2  [corticosteroids, retinoids, or immunosuppress...   
3  [support groups may provide assistance to the ...   

                                            response  \
0  The protocol for managing sepsis in a critical...   
1  Common symptoms of appendicitis include abdomi...   
2  Effective treatments for sudden patchy hair lo...   
3  Treatment for brain injury may include:\n\n1. ...   

                                           reference  answer_relevancy  \
0  What is the protocol for managing sepsis in a ...          0.976425   
1  What a

In [68]:
df_hp1 = all_evaluation_results['prompt_engg_responses_hp1']
df_hp1

,user_input,retrieved_contexts,response,reference,answer_relevancy,llm_context_precision_without_reference,faithfulness
0,What is the protocol for managing sepsis in a ...,[Parenteral antibiotics should be given after ...,Managing sepsis in a critical care unit involv...,What is the protocol for managing sepsis in a ...,1.000000,0.700000,0.409091
1,"What are the common symptoms for appendicitis,...",[Appendicitis\nAppendicitis is acute inflammat...,Common symptoms of appendicitis include:\n\n1....,"What are the common symptoms for appendicitis,...",0.919637,1.000000,0.800000
2,What are the effective treatments or solutions...,"[corticosteroids, retinoids, or immunosuppress...","Sudden patchy hair loss, often referred to as ...",What are the effective treatments or solutions...,0.946577,1.000000,0.750000
3,What treatments are recommended for a person w...,[support groups may provide assistance to the ...,Treatment for a person who has sustained a phy...,What treatments are recommended for a person w...,0.948083,0.583333,0.631579


In [69]:
df_hp2 = all_evaluation_results['prompt_engg_responses_hp2']
df_hp2

,user_input,retrieved_contexts,response,reference,answer_relevancy,llm_context_precision_without_reference,faithfulness
0,What is the protocol for managing sepsis in a ...,[Parenteral antibiotics should be given after ...,Sepsis is a life-threatening condition that ar...,What is the protocol for managing sepsis in a ...,0.988421,1.000000,0.468750
1,"What are the common symptoms for appendicitis,...",[Appendicitis\nAppendicitis is acute inflammat...,Appendicitis is an inflammation of the appendi...,"What are the common symptoms for appendicitis,...",0.920255,1.000000,0.650000
2,What are the effective treatments or solutions...,"[corticosteroids, retinoids, or immunosuppress...","Sudden patchy hair loss, often referred to as ...",What are the effective treatments or solutions...,0.933219,0.887500,0.320000
3,What treatments are recommended for a person w...,[support groups may provide assistance to the ...,When a person sustains a physical injury to br...,What treatments are recommended for a person w...,0.904829,0.638889,0.571429


In [70]:
df_hp3 = all_evaluation_results['prompt_engg_responses_hp3']
df_hp3

,user_input,retrieved_contexts,response,reference,answer_relevancy,llm_context_precision_without_reference,faithfulness
0,What is the protocol for managing sepsis in a ...,[Parenteral antibiotics should be given after ...,The protocol for managing sepsis in a critical...,What is the protocol for managing sepsis in a ...,1.000000,0.700000,0.461538
1,"What are the common symptoms for appendicitis,...",[Appendicitis\nAppendicitis is acute inflammat...,Common symptoms of appendicitis include:\n\n1....,"What are the common symptoms for appendicitis,...",0.934959,1.000000,0.833333
2,What are the effective treatments or solutions...,"[corticosteroids, retinoids, or immunosuppress...",**Key Steps for Addressing Sudden Patchy Hair ...,What are the effective treatments or solutions...,0.907836,1.000000,0.521739
3,What treatments are recommended for a person w...,[support groups may provide assistance to the ...,The treatment for a person who has sustained a...,What treatments are recommended for a person w...,0.942174,0.583333,0.764706


In [71]:
df_hp4 = all_evaluation_results['prompt_engg_responses_hp4']
df_hp4

,user_input,retrieved_contexts,response,reference,answer_relevancy,llm_context_precision_without_reference,faithfulness
0,What is the protocol for managing sepsis in a ...,[Parenteral antibiotics should be given after ...,Managing sepsis in a critical care unit is cru...,What is the protocol for managing sepsis in a ...,0.975360,0.200000,0.538462
1,"What are the common symptoms for appendicitis,...",[Appendicitis\nAppendicitis is acute inflammat...,Sure! Let’s break this down.\n\n### Common Sym...,"What are the common symptoms for appendicitis,...",0.917404,1.000000,0.789474
2,What are the effective treatments or solutions...,"[corticosteroids, retinoids, or immunosuppress...","Sudden patchy hair loss, often referred to as ...",What are the effective treatments or solutions...,0.933186,1.000000,0.400000
3,What treatments are recommended for a person w...,[support groups may provide assistance to the ...,When treating a person who has sustained a phy...,What treatments are recommended for a person w...,0.948107,0.588889,0.461538


In [72]:
df_hp5 = all_evaluation_results['prompt_engg_responses_hp5']
df_hp5

,user_input,retrieved_contexts,response,reference,answer_relevancy,llm_context_precision_without_reference,faithfulness
0,What is the protocol for managing sepsis in a ...,[Parenteral antibiotics should be given after ...,The protocol for managing sepsis in a critical...,What is the protocol for managing sepsis in a ...,0.976425,0.700000,0.227273
1,"What are the common symptoms for appendicitis,...",[Appendicitis\nAppendicitis is acute inflammat...,Common symptoms of appendicitis include abdomi...,"What are the common symptoms for appendicitis,...",0.928503,1.000000,0.909091
2,What are the effective treatments or solutions...,"[corticosteroids, retinoids, or immunosuppress...",Effective treatments for sudden patchy hair lo...,What are the effective treatments or solutions...,0.949588,0.700000,0.692308
3,What treatments are recommended for a person w...,[support groups may provide assistance to the ...,Treatment for brain injury may include:\n\n1. ...,What treatments are recommended for a person w...,0.930236,0.638889,0.615385


### 👉  **Observations**



*   The answers for all 4 questions look like relevant but the faithfulness is still not upto the mark with prompt engineering based model. Still producing additional contents.  
*   Among all the 5 hyper parameters the 5th hyper parameter set ("Give a concise answer", "k": 4, "max_tokens": 300, "temperature": 0.2, "top_p": 0.85) is giving better result as compared to other 4. But still not upto the mark as one of the answer is not giving good faithfulness score. Let's see how RAG improves it.



### Evaluation 3: RAG Response Evaluation

In [73]:
all_evaluation_results_RAG = {}

# Loop through each hyperparameter set (hp1 to hp5)
for i in range(1, 6):
    hp_column_name = f"responses_with_RAG_set_{i}"
    print(f"------------ Evaluating {hp_column_name} ------------")

    # Extract responses for the current hyperparameter set
    RAGAS_RAG_responses_with_rag = result_df[hp_column_name].tolist()

    # Create a HuggingFace Dataset for the current set
    ragas_rag_dataset_with_RAG = Dataset.from_dict({
        "question": RAGAS_questions,
        "answer": RAGAS_RAG_responses_with_rag,
        "contexts": RAGAS_contexts,
        "reference": RAGAS_questions
    })

    # Run RAGAS evaluation
    result_with_rag = evaluate(
        ragas_rag_dataset_with_RAG,
        metrics=[
            answer_relevancy,
            context_precision,
            faithfulness,
        ],
        llm=evaluator_llm,
        embeddings=embedding_model
    )

    # Convert results to DataFrame
    df_rag = result_with_rag.to_pandas()

    # Store the DataFrame in the dictionary
    all_evaluation_results_RAG[hp_column_name] = df_rag

    # Print the DataFrame
    print(df_rag)

    # Calculate and print average metrics
    print("\nAverage metrics for current hyperparameter set:")
    print(df_rag.mean(numeric_only=True))
    print("-" * 50)

------------ Evaluating responses_with_RAG_set_1 ------------


Evaluating:   0%|          | 0/12 [00:00<?, ?it/s]

                                          user_input  \
0  What is the protocol for managing sepsis in a ...   
1  What are the common symptoms for appendicitis,...   
2  What are the effective treatments or solutions...   
3  What treatments are recommended for a person w...   

                                  retrieved_contexts  \
0  [Parenteral antibiotics should be given after ...   
1  [Appendicitis\nAppendicitis is acute inflammat...   
2  [corticosteroids, retinoids, or immunosuppress...   
3  [support groups may provide assistance to the ...   

                                            response  \
0  The protocol for managing sepsis in a critical...   
1  The common symptoms of appendicitis include ep...   
2  Effective treatments for sudden patchy hair lo...   
3  Initial treatment consists of ensuring a relia...   

                                           reference  answer_relevancy  \
0  What is the protocol for managing sepsis in a ...          1.000000   
1  What a

Evaluating:   0%|          | 0/12 [00:00<?, ?it/s]

                                          user_input  \
0  What is the protocol for managing sepsis in a ...   
1  What are the common symptoms for appendicitis,...   
2  What are the effective treatments or solutions...   
3  What treatments are recommended for a person w...   

                                  retrieved_contexts  \
0  [Parenteral antibiotics should be given after ...   
1  [Appendicitis\nAppendicitis is acute inflammat...   
2  [corticosteroids, retinoids, or immunosuppress...   
3  [support groups may provide assistance to the ...   

                                            response  \
0  The protocol for managing sepsis in a critical...   
1  Common symptoms of appendicitis include epigas...   
2  Effective treatments for sudden patchy hair lo...   
3  Initial treatment consists of ensuring a relia...   

                                           reference  answer_relevancy  \
0  What is the protocol for managing sepsis in a ...          1.000000   
1  What a

...

                                          user_input  \
0  What is the protocol for managing sepsis in a ...   
1  What are the common symptoms for appendicitis,...   
2  What are the effective treatments or solutions...   
3  What treatments are recommended for a person w...   

                                  retrieved_contexts  \
0  [Parenteral antibiotics should be given after ...   
1  [Appendicitis\nAppendicitis is acute inflammat...   
2  [corticosteroids, retinoids, or immunosuppress...   
3  [support groups may provide assistance to the ...   

                                            response  \
0  The protocol for managing sepsis in a critical...   
1  Common symptoms of appendicitis include epigas...   
2  Effective treatments for sudden patchy hair lo...   
3  Initial treatment consists of ensuring a relia...   

                                           reference  answer_relevancy  \
0  What is the protocol for managing sepsis in a ...          1.000000   
1  What a

Evaluating:   0%|          | 0/12 [00:00<?, ?it/s]

                                          user_input  \
0  What is the protocol for managing sepsis in a ...   
1  What are the common symptoms for appendicitis,...   
2  What are the effective treatments or solutions...   
3  What treatments are recommended for a person w...   

                                  retrieved_contexts  \
0  [Parenteral antibiotics should be given after ...   
1  [Appendicitis\nAppendicitis is acute inflammat...   
2  [corticosteroids, retinoids, or immunosuppress...   
3  [support groups may provide assistance to the ...   

                                            response  \
0  The protocol for managing sepsis in a critical...   
1  The common symptoms of appendicitis include ep...   
2  Effective treatments for sudden patchy hair lo...   
3  Initial treatment consists of ensuring a relia...   

                                           reference  answer_relevancy  \
0  What is the protocol for managing sepsis in a ...          1.000000   
1  What a

Evaluating:   0%|          | 0/12 [00:00<?, ?it/s]

                                          user_input  \
0  What is the protocol for managing sepsis in a ...   
1  What are the common symptoms for appendicitis,...   
2  What are the effective treatments or solutions...   
3  What treatments are recommended for a person w...   

                                  retrieved_contexts  \
0  [Parenteral antibiotics should be given after ...   
1  [Appendicitis\nAppendicitis is acute inflammat...   
2  [corticosteroids, retinoids, or immunosuppress...   
3  [support groups may provide assistance to the ...   

                                            response  \
0  The protocol for managing sepsis in a critical...   
1  The common symptoms of appendicitis include ep...   
2  Effective treatments for sudden patchy hair lo...   
3  Initial treatment for a physical injury to bra...   

                                           reference  answer_relevancy  \
0  What is the protocol for managing sepsis in a ...          1.000000   
1  What a

In [74]:
df_RAG1 = all_evaluation_results_RAG['responses_with_RAG_set_1']
df_RAG1

,user_input,retrieved_contexts,response,reference,answer_relevancy,llm_context_precision_without_reference,faithfulness
0,What is the protocol for managing sepsis in a ...,[Parenteral antibiotics should be given after ...,The protocol for managing sepsis in a critical...,What is the protocol for managing sepsis in a ...,1.000000,1.000000,1.0
1,"What are the common symptoms for appendicitis,...",[Appendicitis\nAppendicitis is acute inflammat...,The common symptoms of appendicitis include ep...,"What are the common symptoms for appendicitis,...",0.934959,1.000000,1.0
2,What are the effective treatments or solutions...,"[corticosteroids, retinoids, or immunosuppress...",Effective treatments for sudden patchy hair lo...,What are the effective treatments or solutions...,0.950697,0.804167,0.7
3,What treatments are recommended for a person w...,[support groups may provide assistance to the ...,Initial treatment consists of ensuring a relia...,What treatments are recommended for a person w...,0.874546,0.638889,1.0


In [75]:
df_RAG2 = all_evaluation_results_RAG['responses_with_RAG_set_2']
df_RAG2

,user_input,retrieved_contexts,response,reference,answer_relevancy,llm_context_precision_without_reference,faithfulness
0,What is the protocol for managing sepsis in a ...,[Parenteral antibiotics should be given after ...,The protocol for managing sepsis in a critical...,What is the protocol for managing sepsis in a ...,1.000000,0.700000,0.71875
1,"What are the common symptoms for appendicitis,...",[Appendicitis\nAppendicitis is acute inflammat...,Common symptoms of appendicitis include epigas...,"What are the common symptoms for appendicitis,...",0.934959,1.000000,1.00000
2,What are the effective treatments or solutions...,"[corticosteroids, retinoids, or immunosuppress...",Effective treatments for sudden patchy hair lo...,What are the effective treatments or solutions...,0.944305,0.700000,1.00000
3,What treatments are recommended for a person w...,[support groups may provide assistance to the ...,Initial treatment consists of ensuring a relia...,What treatments are recommended for a person w...,0.872299,0.638889,1.00000


In [76]:
df_RAG3 = all_evaluation_results_RAG['responses_with_RAG_set_3']
df_RAG3

,user_input,retrieved_contexts,response,reference,answer_relevancy,llm_context_precision_without_reference,faithfulness
0,What is the protocol for managing sepsis in a ...,[Parenteral antibiotics should be given after ...,The protocol for managing sepsis in a critical...,What is the protocol for managing sepsis in a ...,1.000000,0.700000,0.655172
1,"What are the common symptoms for appendicitis,...",[Appendicitis\nAppendicitis is acute inflammat...,Common symptoms of appendicitis include epigas...,"What are the common symptoms for appendicitis,...",0.917734,1.000000,1.000000
2,What are the effective treatments or solutions...,"[corticosteroids, retinoids, or immunosuppress...",Effective treatments for sudden patchy hair lo...,What are the effective treatments or solutions...,0.938963,0.700000,0.833333
3,What treatments are recommended for a person w...,[support groups may provide assistance to the ...,Initial treatment consists of ensuring a relia...,What treatments are recommended for a person w...,0.858266,0.679167,1.000000


In [77]:
df_RAG4 = all_evaluation_results_RAG['responses_with_RAG_set_4']
df_RAG4

,user_input,retrieved_contexts,response,reference,answer_relevancy,llm_context_precision_without_reference,faithfulness
0,What is the protocol for managing sepsis in a ...,[Parenteral antibiotics should be given after ...,The protocol for managing sepsis in a critical...,What is the protocol for managing sepsis in a ...,1.000000,0.700000,1.0
1,"What are the common symptoms for appendicitis,...",[Appendicitis\nAppendicitis is acute inflammat...,The common symptoms of appendicitis include ep...,"What are the common symptoms for appendicitis,...",0.934959,1.000000,1.0
2,What are the effective treatments or solutions...,"[corticosteroids, retinoids, or immunosuppress...",Effective treatments for sudden patchy hair lo...,What are the effective treatments or solutions...,0.941299,0.700000,1.0
3,What treatments are recommended for a person w...,[support groups may provide assistance to the ...,Initial treatment consists of ensuring a relia...,What treatments are recommended for a person w...,0.870799,0.638889,1.0


In [78]:
df_RAG5 = all_evaluation_results_RAG['responses_with_RAG_set_5']
df_RAG5

,user_input,retrieved_contexts,response,reference,answer_relevancy,llm_context_precision_without_reference,faithfulness
0,What is the protocol for managing sepsis in a ...,[Parenteral antibiotics should be given after ...,The protocol for managing sepsis in a critical...,What is the protocol for managing sepsis in a ...,1.000000,0.7,1.0
1,"What are the common symptoms for appendicitis,...",[Appendicitis\nAppendicitis is acute inflammat...,The common symptoms of appendicitis include ep...,"What are the common symptoms for appendicitis,...",0.917734,1.0,1.0
2,What are the effective treatments or solutions...,"[corticosteroids, retinoids, or immunosuppress...",Effective treatments for sudden patchy hair lo...,What are the effective treatments or solutions...,0.945753,0.7,1.0
3,What treatments are recommended for a person w...,[support groups may provide assistance to the ...,Initial treatment for a physical injury to bra...,What treatments are recommended for a person w...,0.926636,0.5,1.0


### 👉  **Observations**

*   Using RAGAS evaluation (Faithfulness, Relevancy & Context Precision) also it is observed that the most efficient method is RAG based response.


*   From the 5 different hyper parameter based RAGss it is observed that the first 2 parameters sets ({'k': 3, 'max_tokens': 300, 'temperature': 0.2, 'top_p': 0.9} and
    {'k': 5, 'max_tokens': 400, 'temperature': 0.3, 'top_p': 0.95}) are nearly producing same result with minimal difference.
*   The last parameter set ({'k': 10, 'max_tokens': 700, 'temperature': 0.7, 'top_p': 0.9}) is producing better relevance and faithfulness but less context precision because of high temperature and maximum tokens.


*   We can do some more options of hyper parameter tuning to come up with better result but for now we can proceed with the option 2 ({'k': 5, 'max_tokens': 400, 'temperature': 0.3, 'top_p': 0.95})





### ✍  **Actionable Insights and Business Recommendations**

* This results a Retrieval-Augmented Generation (RAG) based application using an **OpenAI model** that can perform Q\&A from documents for accurate information retrieval.

  * First, I chunked the data to create multiple splits with overlaps.
  * Then I used embedding models to encode the different data splits.
  * Then I stored these embeddings in a vector database.
  * Then I defined the OpenAI model that would take the user query and relevant context via the encoded data chunks.
  * Finally, assembled all these components to build the RAG-based system.
* I've also evaluated the output of a RAG-based system using the **RAGAS framework**, which measures groundedness, relevance, and answer correctness.
* Lastly, I also compared the output from an OpenAI model alone and that from an RAG-based system and understood the benefits of RAG in terms of faithfullness, answer relevancy and context precision.
* For real life scenario we can go for different LLMs testing, more extensive testing with greater samples, more hyper parameter tuning, more refined prompts and do the requried fine tuning for accuracy optimization of this application.

<font size=6 color='#4682B4'>Power Ahead</font>
___